In [2]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import BlockRNNModel
#from darts.callbacks import LossLoggingCallback
from darts.metrics import mae, mape
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tools.callbacks import LossLoggingCallback

In [3]:
# 定義計算95%置信區間的函數
def calculate_95_ci(data):
    lower = np.percentile(data, 2.5)
    upper = np.percentile(data, 97.5)
    return (lower, upper)

In [4]:
# 讀取資料
df = pd.read_csv('../DataSet/EDvisitfileL.csv', encoding='ISO-8859-1')

# 確保 'date' 列是 DateTime 類型並設置為索引
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# 定義訓練集和測試集的結束索引
train_end = 3362  # 根據不同的分類（L, T, Ka, Ke, Y, C）選擇不同的值
# val_end = 3737  # 如果需要驗證集，取消註釋並設置適當的結束索引

# 分割 DataFrame
train_df = df.iloc[:train_end]
test_df = df.iloc[train_end:]

# 步驟2: 使用 MinMaxScaler 縮放數據
scaler = MinMaxScaler()
scaler.fit(train_df[['No']]) 

# 縮放訓練集和測試集
train_df.loc[:, 'No_scaled'] = scaler.transform(train_df[['No']])
test_df.loc[:, 'No_scaled'] = scaler.transform(test_df[['No']])  # 用相同的 scaler 轉換測試集以避免數據泄露

# 創建 Darts 的 TimeSeries 對象
train_series = TimeSeries.from_dataframe(train_df, value_cols='No_scaled')
test_series = TimeSeries.from_dataframe(test_df, value_cols='No_scaled')
combined_series = train_series.concatenate(test_series)

# 原始數據轉換為 TimeSeries 對象（如果需要）
train_series_origin = TimeSeries.from_dataframe(train_df, value_cols='No')
test_series_origin = TimeSeries.from_dataframe(test_df, value_cols='No')

# 選擇需要的列創建多變量時間序列（都是 one hot coding）
columns = ['Dayoff', 'Mon', 'Tue', 'Wed', 'Thr', 'Fri', 'Sat', 'Sun', 'Dayscaled', 'NewYear', '3Lock', 
               'Outbreak','COVID19', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov','Dec']
df_multivariate = df[columns]

# 將 DataFrame 轉換為多變量時間序列
ED_covariates = TimeSeries.from_dataframe(df_multivariate)

# 初始化損失記錄回調
loss_logging_callback = LossLoggingCallback()

# 定義 LSTM 模型
model = BlockRNNModel(
    model='LSTM',
    input_chunk_length=30,
    output_chunk_length=30,  # 對應 forecast_horizon
    hidden_dim=384,
    dropout=0.2887,
    n_rnn_layers=1,
    log_tensorboard=True,
    optimizer_kwargs={'lr': 0.0023},  # 修正學習率設置
    pl_trainer_kwargs={
        "max_epochs": 10,
        "accelerator": "gpu",  # 使用 GPU
        "devices": [0],        # 指定 GPU 設備
        "callbacks": [loss_logging_callback],
    },
)

# 設定 historical_forecasts 參數
forecast_horizon = 30  # 預測30天，即一個月
stride = 30            # 每30天進行一次預測，確保每月初預測
retrain = 90        # 每次預測前重新訓練模型
train_length = 1460    # 使用前1460天（約4年）的數據進行訓練

# 執行 historical_forecasts 以獲取預測結果
forecast_series_list = model.historical_forecasts(
    series=combined_series,
    past_covariates=ED_covariates,
    forecast_horizon=forecast_horizon,
    stride=stride,
    retrain=retrain,
    start=pd.Timestamp('2018-01-01'),
    train_length=train_length,
    last_points_only=False,  # 保留整個月的預測結果
    verbose=True,
)



C:\Users\ian11\AppData\Local\Temp\ipykernel_19504\1020833687.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.loc[:, 'No_scaled'] = scaler.transform(train_df[['No']])
C:\Users\ian11\AppData\Local\Temp\ipykernel_19504\1020833687.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.loc[:, 'No_scaled'] = scaler.transform(test_df[['No']])  # 用相同的 scaler 轉換測試集以避免數據泄露
`enable_optimization=True` is ignored because `retrain` is not `False` or `0`.To hide this warning, set `show_warnings=False` 

  0%|          | 0/57 [00:00<?, ?it/s]

C:\Users\ian11\anaconda3\envs\EDforecast\lib\site-packages\torch\nn\modules\rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2887 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | cri

Training: |                                                                                      | 0/? [00:00<…

Epoch 0: train_loss=0.00542496537799848
Epoch 1: train_loss=0.009161354341960502
Epoch 2: train_loss=0.008754782897433954
Epoch 3: train_loss=0.0037209242664616934
Epoch 4: train_loss=0.0030942953603730647
Epoch 5: train_loss=0.005215473612914797
Epoch 6: train_loss=0.008360227201138767
Epoch 7: train_loss=0.006727812655338747
Epoch 8: train_loss=0.005727342312211531


...

Predicting: |                                                                                    | 0/? [00:00<…

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                                                                    | 0/? [00:00<…

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                                                                    | 0/? [00:00<…

In [5]:
if isinstance(forecast_series_list, list):
    # 方法 1：逐步合併
    # forecast_series = reduce(lambda x, y: x.concatenate(y), forecast_series_list)
    
    # 方法 2：使用 pandas DataFrame 合併
    forecast_df = pd.DataFrame()
    for ts in forecast_series_list:
        temp_df = ts.pd_dataframe()
        forecast_df = pd.concat([forecast_df, temp_df])
    
    # 確保按時間順序排序
    forecast_df = forecast_df.sort_index()
    
    # 去除重複的時間步，保留最後一個出現的值
    forecast_df = forecast_df[~forecast_df.index.duplicated(keep='last')]
    
    # 創建單一的 TimeSeries 對象
    forecast_series = TimeSeries.from_dataframe(forecast_df)
else:
    forecast_series = forecast_series_list

<TimeSeries (DataArray) (date: 1710, component: 1, sample: 1)>
array([[[0.19888642]],

       [[0.15610004]],

       [[0.1337164 ]],

       ...,

       [[0.30983854]],

       [[0.2624231 ]],

       [[0.23393044]]])
Coordinates:
  * date       (date) datetime64[ns] 2018-01-01 2018-01-02 ... 2022-09-06
  * component  (component) object 'No_scaled'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [ ]:
if isinstance(forecast_series_list, list):
    # 方法 1：逐步合併
    # forecast_series = reduce(lambda x, y: x.concatenate(y), forecast_series_list)
    
    # 方法 2：使用 pandas DataFrame 合併
    forecast_df = pd.DataFrame()
    for ts in forecast_series_list:
        temp_df = ts.pd_dataframe()
        forecast_df = pd.concat([forecast_df, temp_df])
    
    # 確保按時間順序排序
    forecast_df = forecast_df.sort_index()
    
    # 去除重複的時間步，保留最後一個出現的值
    forecast_df = forecast_df[~forecast_df.index.duplicated(keep='last')]
    
    # 創建單一的 TimeSeries 對象
    forecast_series = TimeSeries.from_dataframe(forecast_df)
else:
    forecast_series = forecast_series_list

forecast_series_unscaled = scaler.inverse_transform(forecast_series)

# 反轉縮放後，進行實際值與預測值的比較
plt.figure(figsize=(14, 7))
combined_series_origin.plot(label='Actual')  # 使用原始數據進行繪圖
forecast_series_unscaled.plot(label='Forecast')
plt.legend()
plt.title('Actual vs Forecast (Unscaled)')
plt.show()

# 更新 APE 和 AE 計算，使用反轉縮放的預測值
APEdf = (np.abs((combined_series_origin - forecast_series_unscaled) / combined_series_origin) * 100).pd_dataframe()
APEdf = APEdf.reset_index().melt(id_vars=['time'], var_name='component', value_name='APE')

# 重新計算各年度的 MAE、MAPE 等指標
metrics = {}
ape_2018_2019 = []
ape_2020_2021 = []
ape_2022 = []
ae_2018_2019 = []
ae_2020_2021 = []
ae_2022 = []

for year in range(2018, 2023):
    actual = combined_series_origin.slice(pd.Timestamp(f'{year}-01-01'), pd.Timestamp(f'{year}-12-31'))
    forecast = forecast_series_unscaled.slice(pd.Timestamp(f'{year}-01-01'), pd.Timestamp(f'{year}-12-31'))
    
    # 計算 MAE 和 MAPE
    mae_value = mae(actual, forecast)
    mape_value = mape(actual, forecast)
    
    ape_yearly = APEdf.loc[
        (APEdf['time'] >= f'{year}-01-01') & (APEdf['time'] <= f'{year}-12-31'), 'APE'
    ]
    
    actual_values = actual.values().flatten()
    forecast_values = forecast.values().flatten()
    ae_values = np.abs(actual_values - forecast_values)
    
    if year in [2018, 2019]:
        ape_2018_2019.extend(ape_yearly.tolist())
        ae_2018_2019.extend(ae_values.tolist())
    elif year in [2020, 2021]:
        ape_2020_2021.extend(ape_yearly.tolist())
        ae_2020_2021.extend(ae_values.tolist())
    elif year in [2022]:
        ape_2022.extend(ape_yearly.tolist())
        ae_2022.extend(ae_values.tolist())
    
    ci_95 = calculate_95_ci(ape_yearly)
    
    metrics[year] = {
        "MAE": mae_value,
        "MAPE": mape_value,
        "APE 95% CI": ci_95,
        "AE": ae_values
    }

# 計算組合的指標
ci_95_2018_2019 = calculate_95_ci(ape_2018_2019)
mape_2018_2019 = np.mean(ape_2018_2019)
ae_2018_2019_mean = np.mean(ae_2018_2019)

ci_95_2020_2021 = calculate_95_ci(ape_2020_2021)
mape_2020_2021 = np.mean(ape_2020_2021)
ae_2020_2021_mean = np.mean(ae_2020_2021)

ci_95_2022 = calculate_95_ci(ape_2022)
mape_2022 = np.mean(ape_2022)
ae_2022_mean = np.mean(ae_2022)

ape_all = APEdf['APE'].dropna().values
mape_all = np.mean(ape_all)
ci_all = calculate_95_ci(ape_all)
ae_all = np.mean(np.abs(combined_series_origin.values().flatten() - forecast_series_unscaled.values().flatten()))

# 打印結果
for year, m in metrics.items():
    print(f"Year {year}: MAE = {m['MAE']:.2f}, MAPE = {m['MAPE']:.2f}%, APE 95% CI = ({m['APE 95% CI'][0]:.2f}, {m['APE 95% CI'][1]:.2f})")

print(f"2018-2019 Combined: MAPE = {mape_2018_2019:.2f}%, APE 95% CI = ({ci_95_2018_2019[0]:.2f}, {ci_95_2018_2019[1]:.2f}), MAE = {ae_2018_2019_mean:.2f}")
print(f"2020-2021 Combined: MAPE = {mape_2020_2021:.2f}%, APE 95% CI = ({ci_95_2020_2021[0]:.2f}, {ci_95_2020_2021[1]:.2f}), MAE = {ae_2020_2021_mean:.2f}")
print(f"2022              : MAPE = {mape_2022:.2f}%, APE 95% CI = ({ci_95_2022[0]:.2f}, {ci_95_2022[1]:.2f}), MAE = {ae_2022_mean:.2f}")
print(f"2018-2022 Combined: MAPE = {mape_all:.2f}%, APE 95% CI = ({ci_all[0]:.2f}, {ci_all[1]:.2f}), MAE = {ae_all:.2f}")

In [ ]:
if isinstance(forecast_series_list, list):
    # 方法 1：逐步合併
    # forecast_series = reduce(lambda x, y: x.concatenate(y), forecast_series_list)
    
    # 方法 2：使用 pandas DataFrame 合併
    forecast_df = pd.DataFrame()
    for ts in forecast_series_list:
        temp_df = ts.pd_dataframe()
        forecast_df = pd.concat([forecast_df, temp_df])
    
    # 確保按時間順序排序
    forecast_df = forecast_df.sort_index()
    
    # 去除重複的時間步，保留最後一個出現的值
    forecast_df = forecast_df[~forecast_df.index.duplicated(keep='last')]
    
    # 創建單一的 TimeSeries 對象
    forecast_series = TimeSeries.from_dataframe(forecast_df)
else:
    forecast_series = forecast_series_list


# 繪製實際值與預測值的圖表（可選）
plt.figure(figsize=(14, 7))
combined_series.plot(label='Actual')
forecast_series.plot(label='Forecast')
plt.legend()
plt.title('Actual vs Forecast')
plt.show()

# 計算 APE 並創建 APEdf
APEdf = (np.abs((combined_series - forecast_series) / combined_series) * 100).pd_dataframe()
APEdf = APEdf.reset_index().melt(id_vars=['time'], var_name='component', value_name='APE')

# 初始化 metrics 字典和各年份的 APE 與 AE 列表
metrics = {}
ape_2018_2019 = []
ape_2020_2021 = []
ape_2022 = []
ae_2018_2019 = []
ae_2020_2021 = []
ae_2022 = []

# 定義年份範圍
for year in range(2018, 2023):
    # 從 combined_series 中提取相應年份的實際數據
    actual = combined_series.slice(pd.Timestamp(f'{year}-01-01'), pd.Timestamp(f'{year}-12-31'))
    forecast = forecast_series.slice(pd.Timestamp(f'{year}-01-01'), pd.Timestamp(f'{year}-12-31'))
    
    # 計算 MAE 和 MAPE
    mae_value = mae(actual, forecast)
    mape_value = mape(actual, forecast)
    
    # 計算 APE
    ape_yearly = APEdf.loc[
        (APEdf['time'] >= f'{year}-01-01') & (APEdf['time'] <= f'{year}-12-31'), 'APE'
    ]
    
    # 計算 AE
    actual_values = actual.values().flatten()
    forecast_values = forecast.values().flatten()
    ae_values = np.abs(actual_values - forecast_values)  # Absolute Error
    
    # 根據年份將 APE 和 AE 數據添加到相應列表中
    if year in [2018, 2019]:
        ape_2018_2019.extend(ape_yearly.tolist())
        ae_2018_2019.extend(ae_values.tolist())
    elif year in [2020, 2021]:
        ape_2020_2021.extend(ape_yearly.tolist())
        ae_2020_2021.extend(ae_values.tolist())
    elif year in [2022]:
        ape_2022.extend(ape_yearly.tolist())
        ae_2022.extend(ae_values.tolist())
    
    # 計算95%置信區間
    ci_95 = calculate_95_ci(ape_yearly)
    
    # 存儲該年份的指標
    metrics[year] = {
        "MAE": mae_value,
        "MAPE": mape_value,
        "APE 95% CI": ci_95,
        "AE": ae_values  # 存儲絕對誤差
    }

# 計算各組合的指標
ci_95_2018_2019 = calculate_95_ci(ape_2018_2019)
mape_2018_2019 = np.mean(ape_2018_2019)
ae_2018_2019_mean = np.mean(ae_2018_2019)

ci_95_2020_2021 = calculate_95_ci(ape_2020_2021)
mape_2020_2021 = np.mean(ape_2020_2021)
ae_2020_2021_mean = np.mean(ae_2020_2021)

ci_95_2022 = calculate_95_ci(ape_2022)
mape_2022 = np.mean(ape_2022)
ae_2022_mean = np.mean(ae_2022)

# 計算全部數據的指標
ape_all = APEdf['APE'].dropna().values  # 確保沒有 NaN 值
mape_all = np.mean(ape_all)
ci_all = calculate_95_ci(ape_all)
ae_all = np.mean(np.abs(combined_series.values().flatten() - forecast_series.values().flatten()))

# 打印結果
for year, m in metrics.items():
    print(f"Year {year}: MAE = {m['MAE']:.2f}, MAPE = {m['MAPE']:.2f}%, APE 95% CI = ({m['APE 95% CI'][0]:.2f}, {m['APE 95% CI'][1]:.2f})")

print(f"2018-2019 Combined: MAPE = {mape_2018_2019:.2f}%, APE 95% CI = ({ci_95_2018_2019[0]:.2f}, {ci_95_2018_2019[1]:.2f}), MAE = {ae_2018_2019_mean:.2f}")
print(f"2020-2021 Combined: MAPE = {mape_2020_2021:.2f}%, APE 95% CI = ({ci_95_2020_2021[0]:.2f}, {ci_95_2020_2021[1]:.2f}), MAE = {ae_2020_2021_mean:.2f}")
print(f"2022              : MAPE = {mape_2022:.2f}%, APE 95% CI = ({ci_95_2022[0]:.2f}, {ci_95_2022[1]:.2f}), MAE = {ae_2022_mean:.2f}")
print(f"2018-2022 Combined: MAPE = {mape_all:.2f}%, APE 95% CI = ({ci_all[0]:.2f}, {ci_all[1]:.2f}), MAE = {ae_all:.2f}")

# 可選：繪製實際值與預測值的圖表
plt.figure(figsize=(14, 7))
combined_series.plot(label='Actual')
forecast_series.plot(label='Forecast')
plt.legend()
plt.title('Actual vs Forecast')
plt.show()


In [ ]:
forecast_series